In [23]:
import pandas as pd
import numpy as np
import cvxpy as cp

# Load data
generators = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/generators.csv', index_col=0)
demand_data = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/hourlydemandbynode.csv', index_col=0)
shift_factor_matrix = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/shiftfactormatrix.csv', index_col=0)
line_ratings = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/lineratings.csv', index_col=0)

# Extract generator parameters
gen_nodes = generators['NODE'].astype(int).values  # Generator node IDs
gen_costs = generators['MC'].values  # Marginal costs (£/MWh)
gen_caps = generators['CAP'].values  # Maximum capacities (MW)

# Extract demand node parameters
demand_nodes = demand_data.columns.astype(int).values  # Node IDs with demand
node_demands = demand_data.values  # Hourly demand (168x20 matrix)

# Extract network parameters
all_nodes = shift_factor_matrix.columns.astype(int).values  # All node IDs
line_ids = shift_factor_matrix.index.values  # Transmission line IDs

# Map generator and demand nodes to network indices
gen_indices = {node: np.where(all_nodes == node)[0][0] for node in gen_nodes}
demand_indices = {node: np.where(all_nodes == node)[0][0] for node in demand_nodes}

# Decision variable: Power generation by each generator (MW)
q_supply = cp.Variable((len(gen_nodes), node_demands.shape[0]), nonneg=True)

# Objective function: Minimize total generation cost
objective = cp.Minimize(cp.sum(cp.multiply(gen_costs[:, None], q_supply)))

# Constraints
constraints = []

# (1) Hourly generation equals hourly demand
for t in range(node_demands.shape[0]):  # Loop over hours
    total_generation = cp.sum(q_supply[:, t])  # Total generation at hour t
    total_demand = np.sum(node_demands[t, :])  # Total demand at hour t
    constraints.append(total_generation == total_demand)

# (2) Transmission line constraints
for t in range(node_demands.shape[0]):  # Loop over hours
    # Compute net injection symbolically
    generation_matrix = np.zeros((len(all_nodes), len(gen_nodes)))
    demand_matrix = np.zeros((len(all_nodes), len(demand_nodes)))

    for g, node in enumerate(gen_nodes):
        generation_matrix[gen_indices[node], g] = 1

    for d, node in enumerate(demand_nodes):
        demand_matrix[demand_indices[node], d] = 1

    net_injection = generation_matrix @ q_supply[:, t] - demand_matrix @ node_demands[t, :]

    # Calculate line flows
    line_flows = shift_factor_matrix.values @ net_injection

    # Enforce line ratings
    for l, line_id in enumerate(line_ids):
        line_rating = line_ratings.loc[line_id, 'RATING_MW']
        if not np.isinf(line_rating):  # Ignore infinite ratings
            constraints.append(line_flows[l] <= line_rating)
            constraints.append(line_flows[l] >= -line_rating)

# (3) Generator capacity constraints
for g in range(len(gen_nodes)):
    for t in range(node_demands.shape[0]):
        constraints.append(q_supply[g, t] <= gen_caps[g])  # Capacity constraint

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS, verbose=True)


# Results
if problem.status == cp.OPTIMAL:
    print(f"Optimal dispatch cost: £{problem.value:.2f}")
    q_supply_table = pd.DataFrame(q_supply.value, index=gen_nodes, columns=demand_data.index)
    print("Optimal generator dispatch (MW):")
    print(q_supply_table)
else:
    print("Optimization problem did not solve to optimality.")


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 09 08:13:09 PM: Your problem has 840 variables, 99456 constraints, and 0 parameters.
(CVXPY) Jan 09 08:13:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 09 08:13:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 09 08:13:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 09 08:13:10 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 09 08:13:13 PM: Compiling problem (target solver=ECOS).